In [25]:
import os
import pandas as pd
import numpy as np
from utils import to_21d_vec, get_beat_vector
import re
import sys
sys.path.insert(1, './harmalysis')
from inputRN2Chord import inputRN

In [26]:
script_dir = os.getcwd()
score_rel_path = "../datasets/bhchorale/bach_score_for_vec/chor336_score_A4_trans_vector_pre.krn"
scorepath = os.path.join(script_dir, score_rel_path)


In [28]:
df = pd.read_csv(scorepath, sep="\t", header=None)
pd.set_option('display.max_rows', df.shape[0]+1)
df.columns = ["harm", "root", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]
#df.columns = ["harm", "voice4", "voice3", "voice2", "voice1", "index", "beat", "measure", "key", "meter" ]

In [29]:
#process dataframe, delete bars, score representations, and "rest slice"
df = df[~df['beat'].astype(str).str.startswith(('=','.','*'))]
df

,harm,root,voice4,voice3,voice2,voice1,index,beat,measure,key,meter
1,i,4BB,4e#,4g#,8b#,4gg#,18,4,.,e#,M4/4
4,IIIb,4DD,4B#,4g#,4dd#,4gg#,21,1,=1,e#,M4/4
5,VI,4GG,4c#,4g#,4cc#,8ee#,22,2,=1,e#,M4/4
7,IIIb,4DD,8B#,4g#,4dd#,8gg#,24,3,=1,e#,M4/4
9,III,4DD,4G#,4g#,4dd#,4bb#,26,4,=1,e#,M4/4
11,V7b/-VII,4EE,4c##,4g#,4ee#,4aa#,28,1,=2,e#,M4/4
12,V7/III,4AA,4d#,4f##,8dd#,4aa#,29,2,=2,e#,M4/4
14,III;,4DD;,4G#;,4d#;,4b#;,4gg#;,31,3,=2,e#,M4/4
15,viiob/-VII,4GG#,4E#,4e#,4cc##,4gg#,32,4,=2,e#,M4/4
17,-VII,4AA,4D#,4a#,4dd#,4ff##,34,1,=3,e#,M4/4


In [30]:
prev_note_list = []
for index, row in df.iterrows():
    # for this onset slice 
    this_onset_vec = []
    
    ######################################### Process notes and onsets ########################################
    voices = ["voice4","voice3","voice2","voice1"]
    cur_note_tmp = []
    for v in voices:
        this_note = ''.join(row[[v]].values)
        if ' ' in this_note: #for a part has multiple voices
            this_note_list = this_note.split(" ")
            cur_note_tmp.extend(this_note_list)
        else:
            cur_note_tmp.append(this_note)
    
    cur_note_list = []
    for n in cur_note_tmp:
        note_name = re.sub('[^a-gA-G#-]+', '', n)
        if note_name:
            cur_note_list.append(note_name)
            
    cur_onset_list = list(set(cur_note_list) - set(prev_note_list))
    prev_note_list = cur_note_list
    
    
    note_vec = to_21d_vec(cur_note_list)
    onset_vec = to_21d_vec(cur_onset_list)
    this_onset_vec.extend(note_vec)
    this_onset_vec.extend(onset_vec)
    
    ######################################### Process beat position ########################################
    beat_pos = ''.join(row[["beat"]].values)
    meter = ''.join(row[["meter"]].values)
    meter = meter.replace("M", "")
    beat_vec = get_beat_vector(beat_pos, meter)
    this_onset_vec.extend(beat_vec)
    
    ######################################### Process chord label ########################################
    harm = ''.join(row[["harm"]].values)
    harm = re.sub('[()]', '', harm) # process string
    key = ''.join(row[["key"]].values)
    key_harm = key + ":" + harm
    print(key_harm)
    try:
        chord_label = inputRN(key_harm)["Chord label"]
    except:
        continue # pass this onset slice if RN is not recognizable
    this_onset_vec.append(str(chord_label))
    print(this_onset_vec)
  
    #########################################################################################################


e#:i
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 'E# minor']
e#:IIIb
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 'G# major']
e#:VI
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 'C# major']
e#:IIIb
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 'G# major']
e#:III
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 'G# major']
e#:V7b/-VII
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 'A# dominant seventh']
e#:V7/III
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [12]:
print(inputRN ("A-:IM7")["Chord label"])

Ab major seventh
